In [1]:
import os
import yaml
import glob

In [ ]:
!nvidia-smi

In [ ]:
%env DRIVE_FOLDER=/content/drive/MyDrive
%env PROJECT_FOLDER=/content/drive/MyDrive/ProgettoML
%env SSD_FOLDER=/content/drive/MyDrive/ProgettoML/SSD
%env CYCLEGAN_FOLDER=/content/drive/MyDrive/ProgettoML/CycleGAN
%env ADAIN_FOLDER=/content/drive/MyDrive/ProgettoML/AdaIN
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

#<h1>Baseline SSD</h1>

In [ ]:
os.chdir("/content")

!git clone https://github.com/ekoops/SSD.git
if not os.path.isdir(os.environ["SSD_FOLDER"]):
  !mv SSD $SSD_FOLDER
else:
  !rsync -a ./SSD/ $SSD_FOLDER/
  !rm -r SSD

!pip install -r $SSD_FOLDER/requirements.txt

##<h3>Baseline SSD training</h3>

In [ ]:
os.chdir(os.environ["SSD_FOLDER"])
!python train.py --config-file configs/vgg_ssd300_voc0712.yaml

##<h3>Baseline SSD test on ClipArt1k dataset

In [ ]:
os.chdir(os.environ["SSD_FOLDER"])
!python test.py --config-file configs/vgg_ssd300_clipart.yaml

#<h1>CycleGAN</h1>

In [ ]:
os.chdir("/content")
!git clone https://github.com/ekoops/pytorch-CycleGAN-and-pix2pix.git
if not os.path.isdir(os.environ["CYCLEGAN_FOLDER"]):
  !mv pytorch-CycleGAN-and-pix2pix $CYCLEGAN_FOLDER
else:
  !rsync -a ./pytorch-CycleGAN-and-pix2pix/ $CYCLEGAN_FOLDER/
  !rm -r pytorch-CycleGAN-and-pix2pix

!pip install -r $CYCLEGAN_FOLDER/requirements.txt

##<h3>CycleGAN training</h3>

In [ ]:
%env trainA_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/VOC_trainval_merge.txt
%env trainB_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/CLIPART/clipart/ImageSets/Main/train.txt
os.chdir(os.environ["CYCLEGAN_FOLDER"])
!python train.py --name VOC0712_ClipArt1k --dataroot inputs --display_id -1\
  --verbose --save_latest_freq 500000 --save_epoch_freq 1 --epoch_count 1\
  --n_epochs 10 --n_epochs_decay 10 --lr 0.00001 --continue_train

##<h3>VOC 2007/12 trainval datasets DT to ClipArt1k domain</h3>

In [ ]:
%env testA_FILTER_FILE=/content/drive/MyDrive/ProgettoML/datasets/VOC_trainval_merge.txt
os.chdir(os.environ["CYCLEGAN_FOLDER"])
!python test.py --name VOC0712_ClipArt1k --model test --dataroot inputs/testA\
  --preprocess none --verbose --no_dropout --num_test 16551 --results_dir\
  ../datasets/ --no_real

# Baseline SSD fine-tuning with VOC 2007/12-to-ClipArt1k images

In [ ]:
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC0712_ClipArt1k
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

GUARD = True
startIndex = 1

os.chdir(os.environ["SSD_FOLDER"])
configFileList = [
        "vgg_ssd300_voc0712toclipart_ft", #It: 520
        "vgg_ssd300_voc0712toclipart_ft2", #It: 1100
        "vgg_ssd300_voc0712toclipart_ft3", #It: 10000
]

lastCheckpointFilePath = "outputs/vgg_ssd300_voc0712/last_checkpoint.txt"

if not GUARD:
  for configFile in configFileList:
    folderPath = f"outputs/{configFile}"
    !rm -r $folderPath
    !mkdir $folderPath
    !cp $lastCheckpointFilePath $folderPath/

for idx, configFile in enumerate(configFileList, 1):
  configFilePath = f"configs/{configFile}.yaml"
  if idx >= startIndex:
    !python train.py --config-file $configFilePath

# AdaIN

In [ ]:
os.chdir("/content")

!git clone https://github.com/ekoops/pytorch-AdaIN.git
if not os.path.isdir(os.environ["ADAIN_FOLDER"]):
  !mv pytorch-AdaIN $ADAIN_FOLDER
else:
  !rsync -a ./pytorch-AdaIN/ $ADAIN_FOLDER/
  !rm -r pytorch-AdaIN

!pip install -r $ADAIN_FOLDER/requirements.txt

# Baseline SSD fine-tuning with VOC 2007/12 online style-transferred using a style transfer network based on AdaIN

### Validation

In [ ]:
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

GUARD = False
startIndex = 1

os.chdir(os.environ["SSD_FOLDER"])
configFileList = [
        "vgg_ssd300_voc0712_AdaINst_ft", #TR: 1.0, alpha: 1.0
        "vgg_ssd300_voc0712_AdaINst_ft2", #TR: 0.5, alpha: 1.0
        "vgg_ssd300_voc0712_AdaINst_ft3", #TR: 1.0, alpha: 0.5
        "vgg_ssd300_voc0712_AdaINst_ft4" #TR: 0.5, alpha: 0.5
]
lastCheckpointFilePath = "outputs/vgg_ssd300_voc0712/last_checkpoint.txt"

if not GUARD:
  for configFile in configFileList:
    folderPath = f"outputs/{configFile}"
    !rm -r $folderPath
    !mkdir $folderPath
    !cp $lastCheckpointFilePath $folderPath/

bestConfig = {
  "configFile": "",
  "mAP": 0.0,
}

for idx, configFile in enumerate(configFileList, 1):
  configFilePath = f"configs/{configFile}.yaml"
  if idx >= startIndex:
    !python train.py --config-file $configFilePath --enable_style_transfer

for configFile in configFileList:
  path = f"outputs/{configFile}/inference/clipart_test/*.txt"
  for filename in glob.glob(path):
    with open(filename, 'r') as f:
      line = f.read()
      mAP = float(line.split()[1])
      if mAP > float(bestConfig["mAP"]):
        bestConfig = {
          "configFile": configFile,
          "mAP": mAP
        }

bestConfigFilePath = f"configs/{bestConfig['configFile']}.yaml"
with open(bestConfigFilePath, "r") as f:
  cfg = yaml.load(f)
  bestConfig["transferRatio"] = cfg["ADAIN"]["LOADER"]["TRANSFER_RATIO"]
  bestConfig["alpha"] = cfg["ADAIN"]["MODEL"]["ALPHA"]

print(f"Best configuration:\n{bestConfig}")

## Retraining with the best obtained parameters increasing iterations number

In [ ]:
%env VOC_ROOT=/content/drive/MyDrive/ProgettoML/datasets/VOC
%env CLIPART_ROOT=/content/drive/MyDrive/ProgettoML/datasets/CLIPART

lastCheckpointFilePath = "outputs/vgg_ssd300_voc0712/last_checkpoint.txt"

GUARD = True

os.chdir(os.environ["SSD_FOLDER"])
configFile = "vgg_ssd300_voc0712_AdaINst_ft5"

if not GUARD:
    folderPath = f"outputs/{configFile}"
    !rm -r $folderPath
    !mkdir $folderPath
    !cp $lastCheckpointFilePath $folderPath/
    
    configFilePath = f"configs/{configFile}.yaml"
    !python train.py --config-file $configFilePath --enable_style_transfer
    
    resultsPath = f"outputs/{configFile}/inference/clipart_test/*.txt"
    for filename in glob.glob(resultsPath):
      with open(filename, 'r') as f:
        line = f.read()
        mAP = float(line.split()[1])
        print(f"Obtained mAP: {mAP}")